<a href="https://colab.research.google.com/github/Prkhar05/Tiny_Text2SQL/blob/main/sft_training_pip_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations and Imports

In [ ]:
# !pip install -U -q accelerate==0.27 peft bitsandbytes transformers trl datasets
# transformers[sentencepiece]

In [ ]:
!pip install -U -q accelerate peft bitsandbytes git+https://github.com/huggingface/transformers trl datasets

In [ ]:
import torch
from datasets import load_dataset,Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftConfig, PeftModel,get_peft_model
from trl import SFTTrainer

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Setting the Fine-tuning Parameters

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "PipableAI/pip-sql-1.3b"

# The instruction dataset to use
dataset_name = "Cynaptics/Test2Sql_InstructionTuned"

# Fine-tuned model name
new_model = "sft-pip-sql"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = False

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 5e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.5

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = 100

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = max_steps//5

# Log every X updates steps
logging_steps = max_steps//100

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2800

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Fine-tuning and dataset loading

In [ ]:
dataset = load_dataset(dataset_name,split='small_validation')
# def prompt_pip_sql(d):
#   prompt = f"""<schema>{d['schema']}</schema>
#     <question>{d['question']}</question>
#     <sql>{d['query']}</sql>"""
#   return prompt

# texts=[prompt_pip_sql(d) for d in dataset]
data_proc = Dataset.from_dict({"text" : dataset['prompt']})
data_proc

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating small_validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating small_train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating spider_test split:   0%|          | 0/4840 [00:00<?, ? examples/s]

Generating sql_eval_test split:   0%|          | 0/3509 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/489956 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/400M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/442 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
print(model)

In [ ]:
# print(model.transformer)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
print_trainable_parameters(model)
# model.named_parameters()

trainable params: 132220928 || all params: 739346432 || trainable%: 17.88


In [ ]:
# target_modules=["q_proj", "v_proj"]
# target_modules=["q", "v"]
# target_modules=["query_key_value"]
# target_modules=None
# target_modules="all-linear"

# def find_all_linear_names(model):
#     lora_module_names = set()
#     for name, module in model.named_modules():
#         if isinstance(module, bnb.nn.Linear4bit):
#             names = name.split(".")
#             lora_module_names.add(names[0] if len(names) == 1 else names[-1])

#     if "lm_head" in lora_module_names:  # needed for 16-bit
#         lora_module_names.remove("lm_head")
#     return list(lora_module_names)

# def get_specific_layer_names(model):
#     layer_names = []
#         for name, module in model.named_modules():
#         if isinstance(module, (torch.nn.Linear, torch.nn.Embedding, torch.nn.Conv2d, Conv1D)):
#             layer_names.append('.'.join(name.split('.')[4:]).split('.')[0])

#     return layer_names

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
            "q_proj",
            "v_proj",
            "k_proj",
            "out_proj",
            "fc_in",
            "fc_out",
            "wte",
        ],
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
m=get_peft_model(model,peft_config)
m.print_trainable_parameters()

trainable params: 4,718,592 || all params: 1,351,190,528 || trainable%: 0.34921736810754195


In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # push_to_hub = True,
    )

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data_proc,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer = SFTTrainer(
    model=get_peft_model(model,peft_config),
    train_dataset=data_proc,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
trainer.train()
trainer.model.save_pretrained(new_model)

Step,Training Loss
1,0.907500
2,0.855000
3,0.859400
4,0.906200
5,1.250900
6,0.914100
7,0.903300
8,0.846600
9,0.790300
10,0.779000


In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir results/run

# Merging the adpater weights and the base model

In [ ]:
peft_config = PeftConfig.from_pretrained(f"/content/{new_model}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name ,return_dict=True, torch_dtype=torch.float16, load_in_4bit=True)

In [ ]:
base_model = PeftModel.from_pretrained(base_model,f"/content/{new_model}")
base_model.eval()

base_model = base_model.merge_and_unload()
base_model.save_pretrained("final_model")

In [ ]:
# !huggingface-cli upload ava_ft_nc_full ./final_model

# Evaluation on generated queries

In [ ]:
schema = '''CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    FirstName VARCHAR(50),
    LastName VARCHAR(50),
    Email VARCHAR(100),
    Department VARCHAR(50), #Name of the department
    Position VARCHAR(50),
    Salary DECIMAL(10, 2),
    HireDate DATE
);'''
question="Write a query to get department name and number of employees in them from the table above"
# SELECT Department ,  COUNT(*) FROM Employees GROUP BY Department

In [ ]:
def prompt_pipsql(schema,question):
    return f'''<schema>{schema}</schema>
        <question>{question}</question>
        <sql>'''

In [ ]:
prompt=prompt_pipsql(schema,question)
prompts=[prompt,prompt]

In [ ]:
t=tokenizer(prompts,return_tensors='pt')
output=model.generate(**t,max_length=400)
decode=tokenizer.decode(output[1][len(t.input_ids[1]):],skip_special_tokens=True)
decode

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1465: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
def prompt_pip_sql(d):
  prompt = f"""<schema>{d['schema']}</schema>
    <question>{d['question']}</question>
    <sql>"""
  return prompt
val_dataset=load_dataset('Cynaptics/Test2Sql',split='small_validation')
val_dataset=[prompt_pip_sql(d) for d in val_dataset]
len(val_dataset)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
ans=[]
for i in range(len(val_dataset)):
  token=tokenizer(val_dataset[i],return_tensors='pt', padding=True, truncation=True,max_length=300)
  output=model.generate(**token,max_length=400)
  decode=tokenizer.decode(output[0][len(token.input_ids[0]):],skip_special_tokens=True)
  ans.append(decode)
len(ans)

In [ ]:
input=tokenizer(prompt,return_tensors='pt').to('cuda')
# print(input['input_ids'][0].shape)
output=model.generate(**input,max_length=700)
tokenizer.decode(output[0][len(input.input_ids[0]):],skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


'SELECT movie_title FROM movies WHERE movie_release_year  =  1945 ORDER BY movie_popularity DESC</sql>\n        <sql>SELECT movie_title FROM movies WHERE movie_release_year  =  1945</sql>\n        <sql>SELECT movie_title FROM movies WHERE movie_release_year  =  1945 ORDER BY movie_popularity DESC</sql>\n        <sql>SELECT movie_title FROM movies WHERE movie_release_year  =  1945 ORDER BY movie_popularity DESC</sql>\n        <sql>SELECT movie_title FROM movies WHERE movie_release_year  =  1945 ORDER BY movie_popularity DESC</sql>\n        <sql>SELECT movie_title FROM movies WHERE movie_release_year  =  1945 ORDER BY movie_popularity DESC</sql>\n        <sql>SELECT movie_title FROM movies WHERE movie_release_year  =  1945 ORDER BY movie_popularity DESC</sql>\n'

In [ ]:
# l='What are the research focuses of Miss Xiaohan Yu in ESG, choice under uncertainty, and long-memory?'
# input=tokenizer(l,return_tensors='pt').to('cuda')
# tokenizer.decode(base_model.generate(**input,max_length=8192)[0][len(input.input_ids[0]):],skip_special_tokens=True)

In [ ]:
# new_prompt_list = load_dataset(dataset_name, split="train[150:]")["question"]
# new_prompt_list

In [ ]:
# # Generation of answers
# response_simple=[]
# for l in tqdm.tqdm(new_prompt_list):
#   print(l)
#   input=tokenizer(l,return_tensors='pt').to('cuda')
#   response = tokenizer.decode(base_model.generate(**input,max_length=8192)[0][len(input.input_ids[0]):],skip_special_tokens=True)
#   print('\n')
#   print(response)
#   response_simple.append(response)